In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

# Google Spreadsheet ID
SHEET_ID = '1JDNv_mArl-GPIpxuWS5GxgVEwvjXocS1MrXGc6TYs8M'
SHEET_NAME = 'USD/IDR'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

In [2]:
# Define your custom dataset class
class ForexDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, index):
        x = self.data[index:index+self.sequence_length]  # Select input sequence
        y = self.data[index+self.sequence_length]  # Select target (next value)
        return x, y

# Define the LSTM model
class LSTMForecastModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMForecastModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Use the last time step's output for prediction
        return out

In [3]:
# Load the Forex dataset from a CSV file
dataframe = pd.read_csv(URL)
dataframe.drop('Unnamed: 2', axis=1, inplace=True)

# Convert the preprocessed dataset to a NumPy array
dataset = dataframe['Close'].values.reshape(-1, 1)

# Normalize or scale the dataset if necessary
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

# Split the dataset into training and test sets (adjust as needed)
SPLIT_TRAIN = 0.9
num_splits = int(SPLIT_TRAIN * len(dataset))
train_data = dataset[:num_splits]
test_data = dataset[num_splits:]

# Define the sequence length and create the dataset and data loader
sequence_length = 100  # Number of past time steps to consider for prediction
train_dataset = ForexDataset(train_data, sequence_length)
test_dataset = ForexDataset(test_data, sequence_length)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [4]:
# Define the LSTM model and other hyperparameters
input_size = 1  # Number of input features (single column)
hidden_size = 32
num_layers = 2
output_size = 1  # Number of output values to predict (next value)
model = LSTMForecastModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        inputs = inputs.float()  # Convert inputs to float type
        targets = targets.float()  # Convert targets to float type
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(-1))
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/home/danielbudi/miniconda3/envs/satria-data/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/danielbudi/miniconda3/envs/satria-data/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([14, 1, 1])) that is different to the input size (torch.Size([14, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.0129
Epoch [20/100], Loss: 0.0216
Epoch [30/100], Loss: 0.0283
Epoch [40/100], Loss: 0.0099
Epoch [50/100], Loss: 0.0222
Epoch [60/100], Loss: 0.0038
Epoch [70/100], Loss: 0.0209
Epoch [80/100], Loss: 0.0171
Epoch [90/100], Loss: 0.0179
Epoch [100/100], Loss: 0.0300


In [5]:
# Generate predictions on the test set
model.eval()
future_predictions = []
with torch.no_grad():
    for inputs, _ in test_dataloader:
        inputs = inputs.float()  # Convert inputs to float type
        outputs = model(inputs)
        future_predictions.append(outputs.squeeze().tolist())

future_predictions = np.concatenate(future_predictions)  # Convert list to a single 1D numpy array
future_predictions = future_predictions.reshape(-1, 1)  # Reshape to a 2D array
future_predictions = scaler.inverse_transform(future_predictions)

print('Future Predictions:', future_predictions)

Future Predictions: [[14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54171692]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]
 [14639.54161532]]
